In [38]:
#!conda activate farshid

#!git clone https://github.com/facebookresearch/segment-anything.git
#!cd segment-anything
#!pip install .

#
#!pip uninstall opencv-python
#!pip install opencv-python-headless
#!pip install torch torchvision torchaudio
#!pip install git+https://github.com/facebookresearch/segment-anything.git
#!ollama list
#!ollama pull llava:7b-v1.6

!export OLLAMA_HOST=127.0.0.1:12345
!OLLAMA_METAL=true ollama serve
!export OPENCV_OPENCL_RUNTIME=1
!ollama serve
!lsof -nP -iTCP:11434 | grep LISTEN


Error: listen tcp 127.0.0.1:11434: bind: address already in use
NAME                     ID              SIZE      MODIFIED       
llava:7b-v1.6            8dd30f6b0cb1    4.7 GB    7 minutes ago     
gemma:2b                 b50d6c999e59    1.7 GB    12 minutes ago    
llava:latest             8dd30f6b0cb1    4.7 GB    2 months ago      
reflection:latest        795e433dda61    39 GB     4 months ago      
deepseek-coder:latest    3ddd2d3fc8d2    776 MB    4 months ago      
llama3.1:latest          75382d0899df    4.7 GB    5 months ago      
pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 170370233dd5... 100% ▕████████████████▏ 4.1 GB                         
pulling 72d6f08a42f6... 100% ▕████████████████▏ 624 MB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB        

In [49]:
import cv2
import base64
import requests
import json
import time
import io
import numpy as np
from PIL import Image
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

# Load the SAM model
sam_checkpoint = "/path/to/sam_vit_h_4b8939.pth"  # Ensure this path is correct
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
mask_generator = SamAutomaticMaskGenerator(sam)

def analyze_frame(b64):
    url = "http://localhost:11434/api/generate"
    prompt = "Please analyze this object's appearance and actions in detail."
    data = {
        "model": "llava:7b-v1.6",
        "prompt": prompt,
        "images": [b64],
        "stream": False
    }
    try:
        response = requests.post(url, json=data)
        response_json = response.json()
        if "error" in response_json:
            return f"Error: {response_json['error']}"
        return response_json.get("response", "No response")
    except Exception as e:
        return f"Error: {e}"

def frame_to_base64(frame):
    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(frame_rgb)
    
    # Resize the image if necessary
    max_size = 1024
    if image.width > max_size or image.height > max_size:
        ratio = max_size / max(image.width, image.height)
        new_size = (int(image.width * ratio), int(image.height * ratio))
        image = image.resize(new_size, Image.LANCZOS)
    
    # Convert the image to base64
    buffer = io.BytesIO()
    image.save(buffer, format="JPEG", quality=85)
    return base64.b64encode(buffer.getvalue()).decode("utf-8")

def track_object(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    output = []
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process every second (assuming fps is the frame rate)
        if frame_count % int(fps) == 0:
            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Generate masks
            masks = mask_generator.generate(frame_rgb)
            
            if masks:
                # Get the largest mask
                largest_mask = max(masks, key=lambda x: x["area"])["segmentation"]
                
                # Convert frame to base64
                frame_b64 = frame_to_base64(frame)
                
                # Analyze the frame
                analysis = analyze_frame(frame_b64)
                
                # Store the result
                output.append({
                    "frame": frame_count,
                    "timestamp": frame_count / fps,
                    "analysis": analysis
                })
            
            time.sleep(1)  # Optional: Add delay to avoid overloading the system
        
        frame_count += 1
    
    cap.release()
    return output

# Path to the video file
video_path = "/Users/farshid/projects/videos-iran-old-image-arosi/2024-07-09 11:09:37.228/mimos-devbox-11.MOV"

# Process the video
results = track_object(video_path)

# Save the results to a JSON file
with open("video_analysis.json", "w") as f:
    json.dump(results, f, indent=4)

# Print the results
for result in results:
    print(f"\nFrame {result['frame']} (Timestamp: {result['timestamp']:.2f}s)\nAnalysis: {result['analysis']}")

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/sam_vit_h_4b8939.pth'

In [53]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
print("Segment Anything library is installed and imported successfully!")

Segment Anything library is installed and imported successfully!


In [41]:
# working video
import cv2,base64,requests,json,time
from PIL import Image
import io
def extract_frame_info(frame_base64):
    url="http://localhost:11434/api/generate"
    prompt="""Please analyze this frame and describe:
    1. What's happening in this frame
    2. Any visible text (OCR)
    3. Key objects and people
    4. Actions and movements
    5. Environmental details
    Be very specific and detailed."""
    payload={"model":"llava:7b-v1.6","prompt":prompt,"images":[frame_base64],"stream":False}
    try:
        r=requests.post(url,json=payload)
        j=r.json()
        if"error"in j:return f"Error: {j['error']}"
        return j.get("response","No response received")
    except Exception as e:
        return f"Error processing frame: {e}"
def frame_to_base64(frame):
    fr=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    im=Image.fromarray(fr)
    mx=1024
    if im.width>mx or im.height>mx:
        ratio=mx/max(im.width,im.height)
        ns=(int(im.width*ratio),int(im.height*ratio))
        im=im.resize(ns,Image.LANCZOS)
    b=io.BytesIO()
    im.save(b,format="JPEG",quality=85)
    return base64.b64encode(b.getvalue()).decode("utf-8")
def analyze_video(v):
    c=cv2.VideoCapture(v)
    f=c.get(cv2.CAP_PROP_FPS)
    fc=int(c.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Video FPS: {f}")
    print(f"Total frames: {fc}")
    a=[]
    n=0
    while c.isOpened():
        r,fr=c.read()
        if not r:break
        if n%int(f*60)==0:
            print(f"Processing frame {n}")
            b=frame_to_base64(fr)
            an=extract_frame_info(b)
            d={"frame_number":n,"timestamp":n/f,"analysis":an}
            a.append(d)
            time.sleep(1)
        n+=1
    c.release()
    return a
p="/Users/farshid/projects/videos-iran-old-image-arosi/2024-07-09 11:09:37.228/mimos-devbox-11.MOV"
r=analyze_video(p)
with open("video_analysis.json","w")as f:
    json.dump(r,f,indent=4)
for fr in r:
    print(f"\nFrame {fr['frame_number']} (Timestamp: {fr['timestamp']:.2f}s)")
    print("Analysis:",fr["analysis"])

Video FPS: 29.982395160926284
Total frames: 1107
Processing frame 0

Frame 0 (Timestamp: 0.00s)
Analysis:  In the image, there is a focus on a gaming computer setup with a monitor displaying the startup screen of a computer system. On this screen, we can see the logo for GeForce and the text "GeForce GTX 1650" at the top, followed by the Nvidia logo at the bottom center. Below the Nvidia logo, there are three rows of text. The first row reads "GeForce GTX," followed by "1650" below it. The second row contains "GeForce GTX" again, and the third row also repeats the "GeForce GTX" text, but with a different font size and style than the other two rows.

The monitor is situated on a desk, which has various items on it. To the left of the monitor, there appears to be a keyboard with visible keys such as "Shift," "Ctrl," "Alt," and some letters and numbers. Below the monitor, there are several graphics cards (GPUs) in a vertical arrangement, including at least one GeForce GTX 1650 card. Each 

In [32]:
# working
import requests
url = "http://localhost:11434/api/generate"
payload = {
   "model": "llava:7b-v1.6", 
   "prompt": "Hello from the llava:7b-v1.6 model!",
   "stream": False
}
response = requests.post(url, json=payload)
print("Model response:", response.json()["response"])

Model response:  Hello there! It's nice to meet you. How can I assist you today? 


In [33]:
import requests
import json
from typing import Optional

def display_cities(cities: list[str], preferences: Optional[str] = None):
   return cities

prompt_text = (
   "You are a helpful assistant. A user wants to take a ski trip with their family "
   "but isn't sure where to go. Return a JSON snippet with two keys: 'preferences' "
   "set to 'skiing' and 'cities' set to a list of recommended ski destinations. "
   "Use this format:\n"
   "{\n"
   "  \"function_call\": \"display_cities\",\n" 
   "  \"arguments\": {\n"
   "    \"preferences\": \"...\",\n"
   "    \"cities\": [\"...\", \"...\", \"...\"]\n"
   "  }\n"
   "}\n"
   "Only return valid JSON, no extra commentary."
)

url = "http://localhost:11434/api/generate"
payload = {
   "model": "llava:7b-v1.6",
   "prompt": prompt_text,
   "stream": False
}

response = requests.post(url, json=payload)
model_output = response.json()["response"]
print("----- RAW MODEL OUTPUT -----")
print(model_output)
print("--------------------------------\n")

# Clean up the model output to extract just the JSON part
json_start = model_output.find('{')
json_end = model_output.rfind('}') + 1
clean_json = model_output[json_start:json_end].replace(';', ',')

function_data = json.loads(clean_json)
function_name = function_data["function_call"]
function_args = function_data["arguments"]

# Convert single string of cities to list
if isinstance(function_args["cities"], str):
   function_args["cities"] = [city.strip() for city in function_args["cities"][1:-1].split(',')]

recommended_cities = display_cities(
   cities=function_args["cities"],
   preferences=function_args["preferences"]
)

print(f"Function Name: {function_name}")
print(f"Preferences: {function_args['preferences']}")
print(f"Recommended Cities: {recommended_cities}")

----- RAW MODEL OUTPUT -----
 ```json
{
  "function_call": "display_cities",
  "arguments": {
    "preferences": "skiing",
    "cities": ["Aspen, CO", "Vail, CO", "Park City, UT"]
  }
}
``` 
--------------------------------

Function Name: display_cities
Preferences: skiing
Recommended Cities: ['Aspen, CO', 'Vail, CO', 'Park City, UT']


In [ ]:
import cv2
import base64
import requests
import json
from PIL import Image
import io
import time

def extract_frame_info(frame_base64):
   url = "http://localhost:11434/api/generate"
   prompt = """Please analyze this frame and describe:
   1. What's happening in this frame
   2. Any visible text (OCR)
   3. Key objects and people
   4. Actions and movements
   5. Environmental details
   Be very specific and detailed."""
   
   payload = {
       "model": "llava:7b-v1.6",
       "prompt": prompt,
       "images": [{"data": frame_base64}],
       "stream": False
   }
   
   try:
       response = requests.post(url, json=payload)
       response_json = response.json()
       if 'error' in response_json:
           return f"Error: {response_json['error']}"
       return response_json.get('response', 'No response received')
   except Exception as e:
       return f"Error processing frame: {str(e)}"

def frame_to_base64(frame):
   frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
   pil_image = Image.fromarray(frame_rgb)
   
   # Resize image if too large
   max_size = 1024
   if pil_image.width > max_size or pil_image.height > max_size:
       ratio = max_size / max(pil_image.width, pil_image.height)
       new_size = (int(pil_image.width * ratio), int(pil_image.height * ratio))
       pil_image = pil_image.resize(new_size, Image.LANCZOS)
   
   buffer = io.BytesIO()
   pil_image.save(buffer, format="JPEG", quality=85)
   return base64.b64encode(buffer.getvalue()).decode('utf-8')

def analyze_video(video_path):
   cap = cv2.VideoCapture(video_path)
   fps = cap.get(cv2.CAP_PROP_FPS)
   frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
   
   print(f"Video FPS: {fps}")
   print(f"Total frames: {frame_count}")
   
   frame_analysis = []
   frame_number = 0
   
   while cap.isOpened():
       ret, frame = cap.read()
       if not ret:
           break
           
       if frame_number % int(fps*60) == 0:  # Process 1 frame per second
           print(f"Processing frame {frame_number}")
           frame_base64 = frame_to_base64(frame)
           analysis = extract_frame_info(frame_base64)
           
           frame_data = {
               "frame_number": frame_number,
               "timestamp": frame_number/fps,
               "analysis": analysis
           }
           frame_analysis.append(frame_data)
           time.sleep(1)  # Add delay to prevent rate limiting
           
       frame_number += 1
       
   cap.release()
   return frame_analysis

# Process the video
video_path = "/Users/farshid/projects/videos-iran-old-image-arosi/2024-07-09 11:09:37.228/mimos-devbox-11.MOV"
results = analyze_video(video_path)

# Save results to file
with open('video_analysis.json', 'w') as f:
   json.dump(results, f, indent=4)

# Print results
for frame in results:
   print(f"\nFrame {frame['frame_number']} (Timestamp: {frame['timestamp']:.2f}s)")
   print("Analysis:", frame['analysis'])

Video FPS: 29.982395160926284
Total frames: 1107
Processing frame 0
Processing frame 29
Processing frame 58
Processing frame 87
Processing frame 116
Processing frame 145
Processing frame 174
Processing frame 203
Processing frame 232
Processing frame 261
Processing frame 290
Processing frame 319
Processing frame 348
Processing frame 377
Processing frame 406
Processing frame 435
Processing frame 464
Processing frame 493
Processing frame 522
Processing frame 551
Processing frame 580
Processing frame 609
Processing frame 638
Processing frame 667
Processing frame 696
Processing frame 725
Processing frame 754
Processing frame 783
Processing frame 812
Processing frame 841
Processing frame 870
Processing frame 899
Processing frame 928
Processing frame 957
Processing frame 986
Processing frame 1015
Processing frame 1044
Processing frame 1073
Processing frame 1102

Frame 0 (Timestamp: 0.00s)
Analysis: Error: json: cannot unmarshal object into Go struct field GenerateRequest.images of type api.Im

# test

In [1]:
import cv2

In [14]:
!lsof -nP -iTCP:11434 | grep LISTEN

ollama  4409 farshid    3u  IPv4 0x404da011551d94ce      0t0  TCP 127.0.0.1:11434 (LISTEN)


In [18]:
kill  ollama

SyntaxError: invalid syntax (3006066127.py, line 1)

In [24]:
!ollama list
!ollama pull gemma:2b
#!ollama serve
!export OLLAMA_LISTEN=127.0.0.1:12345
!ollama serve 
!lsof -nP -iTCP:11411 | grep LISTEN

NAME                     ID              SIZE      MODIFIED       
gemma:2b                 b50d6c999e59    1.7 GB    42 seconds ago    
llava:latest             8dd30f6b0cb1    4.7 GB    2 months ago      
llava:7b-v1.6            8dd30f6b0cb1    4.7 GB    2 months ago      
reflection:latest        795e433dda61    39 GB     4 months ago      
deepseek-coder:latest    3ddd2d3fc8d2    776 MB    4 months ago      
llama3.1:latest          75382d0899df    4.7 GB    5 months ago      
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling c1864a5eb193... 100% ▕████████████████▏ 1.7 GB                         
pulling 097a36493f71... 100% ▕████████████████▏ 8.4 KB                         
pulling 109037bec39c... 100% ▕████████████████▏  136 B                         
pulling 22a838ceb7fb... 100% ▕████████████████▏   84 B                         
pulling 887433b89a90... 100% ▕████████████

In [26]:
import requests

# The prompt you want to use
prompt_text = "Hello from the gemma:2b model!"

# Ollama server endpoint
url = "http://localhost:11411/completions"

# JSON payload specifying the model and the prompt
payload = {
    "model": "gemma:2b",
    "prompt": prompt_text
}

try:
    response = requests.post(url, json=payload)
    data = response.json()
    print("Model response:", data["choices"][0]["text"])
except Exception as e:
    print("Error:", e)

Error: HTTPConnectionPool(host='localhost', port=11411): Max retries exceeded with url: /completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x141010650>: Failed to establish a new connection: [Errno 61] Connection refused'))


In [7]:
import requests
from typing import Optional

# 1. Define your function (identical to your original definition)
def display_cities(cities: list[str], preferences: Optional[str] = None):
    """
    Provides a list of cities based on the user's search query and preferences.
    Args:
        preferences (str): The user's preferences for the search, like skiing,
                           beach, restaurants, bbq, etc.
        cities (list[str]): The list of cities being recommended to the user.
    Returns:
        list[str]: The list of cities being recommended to the user.
    """
    return cities

# 2. Define a prompt that asks the model for suggestions
prompt_text = (
    "You are a helpful assistant. A user wants to take a ski trip with their family "
    "but isn’t sure where to go. Return a JSON snippet with two keys: 'preferences' "
    "set to 'skiing' and 'cities' set to a list of recommended ski destinations. "
    "Use this format:\n"
    "{\n"
    "  \"function_call\": \"display_cities\",\n"
    "  \"arguments\": {\n"
    "    \"preferences\": \"...\",\n"
    "    \"cities\": [\"...\", \"...\", \"...\"]\n"
    "  }\n"
    "}\n"
    "Only return valid JSON, no extra commentary."
)

# 3. Make a POST request to the Ollama server with the gemma:2b model
url = "http://localhost:11411/completions"
payload = {
    "model": "gemma:2b",
    "prompt": prompt_text
}

try:
    response = requests.post(url, json=payload, timeout=180)
    data = response.json()

    # The model's text output (hopefully a JSON string)
    model_output = data.get("choices", [{}])[0].get("text", "")

    print("----- RAW MODEL OUTPUT -----")
    print(model_output)
    print("--------------------------------\n")

    # 4. Parse the JSON portion from the model’s text
    import json

    # Attempt to parse the JSON returned by the model
    function_data = json.loads(model_output)

    function_name = function_data["function_call"]
    function_args = function_data["arguments"]

    # 5. Call the Python function with the extracted arguments
    recommended_cities = display_cities(
        cities=function_args["cities"],
        preferences=function_args["preferences"]
    )

    # 6. Print results
    print(f"Function Name: {function_name}")
    print(f"Preferences: {function_args['preferences']}")
    print(f"Recommended Cities: {recommended_cities}")

except Exception as e:
    print("Error:", e)

Error: HTTPConnectionPool(host='localhost', port=11411): Max retries exceeded with url: /completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x141010890>: Failed to establish a new connection: [Errno 61] Connection refused'))
